# Author John Payne
# Step 1
* Generate a response using the Casseopeia wrapper

In [541]:
import requests as re
import time

def getAPI_key():
    f = open("../api_key.txt", "r")
    return f.read()
api_key = getAPI_key()

In [594]:
def check_status():    
    url = 'https://na1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'
    header_dict = {
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "X-Riot-Token": api_key
    }
    

    return re.get(url, headers = header_dict).status_code

In [595]:
check_status()

200

In [544]:
def get_challenger_player_stats():
    url = 'https://na1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'
    header_dict = {
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "X-Riot-Token": api_key
    }

    
    return re.get(url, headers = header_dict).json()

In [545]:
challenger_ladder = get_challenger_player_stats()

In [546]:
def get_challenger_player_id(player_id):
    
    #Using league_v4 
    url = 'https://na1.api.riotgames.com/lol/summoner/v4/summoners/' + player_id
    header_dict = {
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "X-Riot-Token": api_key
    }
    

        
    return re.get(url, headers = header_dict).json()

In [547]:
def get_matches(puuid):
    url = 'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/' + puuid + '/ids'
    header_dict = {
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "X-Riot-Token": api_key
    }
    
    params_dict = {
        #Only picking games played on patch 12.15, 08-10-2022 to 0-24-2022
        'startTime' : 1660125600,
        'endTime' : 1661335200,
        
        #Only picking ranked 5v5 games
        'queue' : 420,
        'type' : 'ranked',
        
        #Only grabbing the first 50 results
        'start' : 0,
        'count' : 50
    }
    return re.get(url, headers = header_dict, params = params_dict).json()

In [548]:
class Summoner:
    
    def __init__(self, summonerId):
        self.summonerId = summonerId
        
        #Looping through the ladder until a match is found with the input summoner id
        for value in challenger_ladder['entries']:
            if value['summonerId'] == self.summonerId:
                
                #Once correct player is found assign lp, wins, losses, winstreak, and ign to the player
                self.leaguepoints = value['leaguePoints']
                self.wins = value['wins']
                self.losses = value['losses']
                self.hotStreak = value['hotStreak']
                self.summonerName = value['summonerName']
        
        #Finds all the accounts of the player in dictionary format
        self.accounts = get_challenger_player_id(summonerId)
        #Finds the puuid of the player
        self.puuid = self.accounts['puuid']
        #Finds the players summoner level
        self.summonerLevel = self.accounts['summonerLevel']
        
        #Returns a list of matches the player has played based on the parameters in the get_matches function
        self.matches = get_matches(self.puuid)
        
        #Total games played by this summoner on this specific patch
        self.total_matches = len(self.matches)
        
    

In [549]:
player_info = {}
for i in range(5):
    temp = 'summoner' + str(i)
    player_info[temp] = Summoner(challenger_ladder['entries'][i]['summonerId'])
    print(i)
    time.sleep(1)

0
1
2
3
4


In [551]:
def get_match_info(match):
    url = 'https://americas.api.riotgames.com/lol/match/v5/matches/' + match
    header_dict = {
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "X-Riot-Token": api_key
    }
    
    return re.get(url, headers = header_dict).json()

In [603]:
class Match_details:
    
    def __init__(self, Match_id):
        self.Match_id = Match_id
        
        #Calling matchesV5 api
        self.call = get_match_info(Match_id)
        
        #Storing api call and filtering out uneeded data
        self.Match_info = self.call['info']
        
        #list of dictionaries including stats for each player
        self.players = self.Match_info['participants']
        
        #Tracking team information
        self.team1 = self.Match_info['teams'][0]
        self.team2 = self.Match_info['teams'][1]
        
        #Storing winning and losing team
        self.team1_win = self.team1['win']
        self.team2_win = self.team2['win']
        
        #Storing bans for each team
        self.team2_bans = self.team1['bans']
        self.team2_bans = self.team1['bans']
      
        #Creating a list of encrypted summoner ids
        self.summoner_Id = []
        for i, player in enumerate(self.players):
            self.summoner_Id.append(player['summonerId'])
        
        #Creating a player class to hold data on each player in the game
        class Player:
    
            def __init__(self, summoner_Id, players):
                self.summoner_Id = summoner_Id
                # Holds lane, champion and team of given player
                self.players = players
                for value in self.players:
                    if value['summonerId'] == self.summoner_Id:
                        self.Lane = value['individualPosition']
                        self.championId = value['championId'] 
                        self.teamId = value['teamId']
                        if self.teamId == 100:
                            self.teamId = 'team1'
                        elif self.teamId == 200:
                            self.teamId = 'team2'
                #Grabbing winrate/winstreak/rank/lp of the given player        
                self.player_stats = get_player_stats(self.summoner_Id)
                        
        
        #Creating a dictionary to hold all the player information (lane, champion, team)
        self.player_position = {}
        for i, summoner_Id in enumerate(self.summoner_Id):
            temp = 'Player' + str(i)
            self.player_position[temp] = Player(summoner_Id, self.players)
            

In [598]:
#Getting matches for each player listed in player_info
games = []
for key in player_info:
    games.append(player_info[key].matches)

In [599]:
#Joining together the sublists (games of each player)
total_games = []
for a in range(len(games)):
    for b in range (len(games[a])):
        total_games.append(games[a][b])

In [600]:
#Finding unique games
total_games = list(set(total_games))

In [601]:
#API request to get player winrates etc.
def get_player_stats(summoner_id):
    url = 'https://na1.api.riotgames.com/lol/league/v4/entries/by-summoner/' + summoner_id
    header_dict = {
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "X-Riot-Token": api_key
    }
    return re.get(url, headers = header_dict).json()

In [604]:
%%time
games = {}
for i, game in enumerate(total_games):  
    temp = 'Game' + str(i)
    games[temp] = Match_details(game)
    print(i)
    time.sleep(22)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
CPU times: total: 19.1 s
Wall time: 47min 39s


In [596]:
len(total_games)

109